<a href="https://colab.research.google.com/github/nigoda/machine_learning/blob/main/20_CNNs_Part_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Convolutional Neural Network(CNNs)**

In [ ]:
!pip uninstall -y tensorflow-gpu
!pip install tensorflow-gpu

Uninstalling tensorflow-gpu-2.0.0b1:
  Successfully uninstalled tensorflow-gpu-2.0.0b1
     |████████████████████████████████| 454.3MB 41kB/s 
     |████████████████████████████████| 4.0MB 29.9MB/s 
     |████████████████████████████████| 4.0MB 24.1MB/s 
     |████████████████████████████████| 6.0MB 29.9MB/s 
     |████████████████████████████████| 471kB 33.6MB/s 
     |████████████████████████████████| 1.2MB 30.4MB/s 
     |████████████████████████████████| 4.9MB 29.2MB/s 
ERROR: tensorflow 2.4.1 has requirement gast==0.3.3, but you'll have gast 0.4.0 which is incompatible.
ERROR: tensorflow 2.4.1 has requirement grpcio~=1.32.0, but you'll have grpcio 1.34.1 which is incompatible.
ERROR: tensorflow 2.4.1 has requirement h5py~=2.10.0, but you'll have h5py 3.1.0 which is incompatible.
ERROR: tensorflow 2.4.1 has requirement tensorflow-estimator<2.5.0,>=2.4.0, but you'll have tensorflow-estimator 2.5.0 which is incompatible.
  Found existing installation: grpcio 1.32.0
    Uninstalling g

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf
import datetime, os

from tensorflow.keras import datasets, layers, models


In [ ]:
(train_images, train_labels), (test_images, test_labels) = datasets.mnist.load_data()

print("Before reshaping:")
print("==================")
print("Number of axes in train_images: ",train_images.ndim)
print("Number of axes in test_images: ",test_images.ndim)
print("Shape of train_images: ",train_images.shape)
print("Shape of test_images: ",test_images.shape)
print()

train_images = train_images.reshape((60000,28,28,1))
test_images = test_images.reshape((10000,28,28,1))

print("After reshaping")
print("================")
print("Number of axes in train_images: ",train_images.ndim)
print("Number of axes in test_images: ",test_images.ndim)
print("Shape of train_images: ",train_images.shape)
print("Shape of test_images: ",test_images.shape)
print()

# Normalize pixel values to be between 0 and 1
train_images, test_images = train_images/255.0, test_images/255.0


Before reshaping:
Number of axes in train_images:  3
Number of axes in test_images:  3
Shape of train_images:  (60000, 28, 28)
Shape of test_images:  (10000, 28, 28)

After reshaping
Number of axes in train_images:  4
Number of axes in test_images:  4
Shape of train_images:  (60000, 28, 28, 1)
Shape of test_images:  (10000, 28, 28, 1)



We print the number of axes and shape of train and test image tensors before and after the reshaping. You can observe that the reshaping operation convert the input 3D tensor into the 4d tensor. An addidtional axes is added in each images.

### **Create the convolution base**

In [ ]:
model = models.Sequential()

The 6 lines of code below the convolutional base using a common pattern: a stack of Conv2D and MaxPooling2D layers.

As input, a CNN takes tensor of shape(image_height,image_width,color_channels), ignoring the batch size. If you are new to color channels, MNIST has one(because the images are grayscale), whereas a color images has three(R,G,B). In this example, we will configure our CNN to process input of shape(28,28,1), which is the format of MNIST images. We do this by passing the argument input_shape to our first layer.

In [ ]:
model.add(layers.Conv2D(32, (3,3), activation='relu', input_shape=(28,28,1)))

In [ ]:
model.add(layers.MaxPooling2D((2,2)))

In [ ]:
model.add(layers.Conv2D(64, (3,3), activation='relu'))

In [ ]:
# Let display the architecture of our model so far
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 9, 9, 64)          36928     
Total params: 55,744
Trainable params: 55,744
Non-trainable params: 0
_________________________________________________________________


To complete our model, we will feed the last output tensor from convolutional base(of shape(3,3,64)) into one or more Dense layers to perform classification. Dense layers take vectors as input (which are 1D), while the current is a 3D tensor. First, we will flatten(or unroll) the 3D output to 1D, then add one or more Dense layers on top. MNIST has 10 output classes, so we use a final Dense layer with 10 outputs and a softmax activation.

In [ ]:
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

In [ ]:
# Here's the complete architecture of our model.

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 9, 9, 64)          36928     
_________________________________________________________________
flatten (Flatten)            (None, 5184)              0         
_________________________________________________________________
dense (Dense)                (None, 64)                331840    
_________________________________________________________________
dense_1 (Dense)              (None, 10)               

### **Compile ans train the model**

In [ ]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(train_images, train_labels, epochs=5)

Epoch 1/5
1875/1875 [==============================] - 81s 43ms/step - loss: 0.1288 - accuracy: 0.9602
Epoch 2/5
1875/1875 [==============================] - 82s 44ms/step - loss: 0.0410 - accuracy: 0.9875
Epoch 3/5
1875/1875 [==============================] - 84s 45ms/step - loss: 0.0284 - accuracy: 0.9912
Epoch 4/5
1875/1875 [==============================] - 85s 45ms/step - loss: 0.0208 - accuracy: 0.9934
Epoch 5/5
1875/1875 [==============================] - 87s 46ms/step - loss: 0.0160 - accuracy: 0.9949


### **Evaluate the model**

In [ ]:
test_loss, test_acc = model.evaluate(test_images, test_labels)

313/313 [==============================] - 4s 12ms/step - loss: 0.0308 - accuracy: 0.9914


### **Classification with Feedforward Neural Networks**


In [ ]:
mnist = tf.keras.datasets.mnist


(x_train, y_train),(x_test, y_test) = mnist.load_data()
x_train, x_test = x_train/255.0, x_test/255.0

In [ ]:
model_dnn = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28,28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10, activation="softmax")
])

model_dnn.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])


In [ ]:
model_dnn.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 128)               100480    
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 10)                1290      
Total params: 101,770
Trainable params: 101,770
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model_dnn.fit(x_train, y_train, epochs=5)

Epoch 1/5
1875/1875 [==============================] - 5s 2ms/step - loss: 0.2958 - accuracy: 0.9139
Epoch 2/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.1417 - accuracy: 0.9582
Epoch 3/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.1059 - accuracy: 0.9677
Epoch 4/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0872 - accuracy: 0.9726
Epoch 5/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0737 - accuracy: 0.9771


In [ ]:
test_loss_dnn, test_acc_dnn = model_dnn.evaluate(x_test, y_test)

313/313 [==============================] - 1s 1ms/step - loss: 0.0786 - accuracy: 0.9769
